**# INSTALLATION**

In [ ]:
# Install required packages

!pip install langchain langchain-openai langchain-community langchain-core requests beautifulsoup4 yfinance pandas numpy python-dotenv tiktoken openai

# IMPORTS

import os
import json
import requests
from datetime import datetime
from typing import Dict, List, Any, Optional, Union
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import BaseTool, tool
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish
from langchain.callbacks.base import BaseCallbackHandler

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-api-key-here"


**# SECTION 1: Agent Architecture Fundamentals (13.2)**

In [ ]:
print("="*60)
print("SECTION 1: AGENT ARCHITECTURE FUNDAMENTALS")
print("="*60)

class AgentCallbackHandler(BaseCallbackHandler):
    """Custom callback handler to track agent reasoning process"""

    def __init__(self):
        self.steps = []
        self.current_step = 0

    def on_agent_action(self, action: AgentAction, **kwargs) -> None:
        self.current_step += 1
        step_info = {
            "step": self.current_step,
            "tool": action.tool,
            "tool_input": action.tool_input,
            "log": action.log
        }
        self.steps.append(step_info)
        print(f"\n🤖 Agent Step {self.current_step}")
        print(f"Tool: {action.tool}")
        print(f"Input: {action.tool_input}")
        print(f"Reasoning: {action.log.split('Action:')[0].strip()}")

    def on_agent_finish(self, finish: AgentFinish, **kwargs) -> None:
        print(f"\n✅ Agent finished with output: {finish.return_values}")

    def get_trace(self) -> List[Dict]:
        return self.steps

# Core Agent Components
class AgentMemory:
    """Simple memory system for agent context"""

    def __init__(self, max_entries: int = 10):
        self.max_entries = max_entries
        self.working_memory = []
        self.long_term_facts = {}

    def add_working_memory(self, entry: Dict[str, Any]):
        """Add entry to working memory with automatic cleanup"""
        self.working_memory.append({
            "timestamp": datetime.now().isoformat(),
            "entry": entry
        })

        # Keep only recent entries
        if len(self.working_memory) > self.max_entries:
            self.working_memory = self.working_memory[-self.max_entries:]

    def add_fact(self, key: str, value: Any):
        """Add persistent fact to long-term memory"""
        self.long_term_facts[key] = {
            "value": value,
            "updated": datetime.now().isoformat()
        }

    def get_context(self) -> str:
        """Get formatted context for agent prompts"""
        context = "Working Memory:\n"
        for mem in self.working_memory[-3:]:  # Last 3 entries
            context += f"- {mem['entry']}\n"

        if self.long_term_facts:
            context += "\nKnown Facts:\n"
            for key, fact in self.long_term_facts.items():
                context += f"- {key}: {fact['value']}\n"

        return context

class AgentPlanner:
    """Simple planning component for multi-step reasoning"""

    def __init__(self, llm):
        self.llm = llm
        self.planning_template = """
        You are an AI planning assistant. Break down the following query into actionable steps.

        Query: {query}
        Available Tools: {tools}

        Create a step-by-step plan. Each step should specify:
        1. What to do
        2. Which tool to use
        3. Expected outcome

        Plan:
        """

    def create_plan(self, query: str, available_tools: List[str]) -> List[Dict]:
        """Create a structured plan for query execution"""
        prompt = self.planning_template.format(
            query=query,
            tools=", ".join(available_tools)
        )

        response = self.llm.invoke(prompt)

        # Parse the response into structured steps
        # In a production system, you'd use more robust parsing
        steps = []
        lines = response.content.split('\n')

        current_step = {}
        for line in lines:
            if line.strip().startswith(('1.', '2.', '3.', '4.', '5.')):
                if current_step:
                    steps.append(current_step)
                current_step = {"description": line.strip()}
            elif line.strip() and current_step:
                if "tool:" in line.lower():
                    current_step["tool"] = line.split(":")[-1].strip()
                elif "outcome:" in line.lower():
                    current_step["expected_outcome"] = line.split(":")[-1].strip()

        if current_step:
            steps.append(current_step)

        return steps

**# SECTION 2: Tool Integration and External APIs (13.4)**

In [ ]:
print("\n" + "="*60)
print("SECTION 2: TOOL INTEGRATION AND EXTERNAL APIS")
print("="*60)

# Custom Tools for Agentic RAG
@tool
def web_search_tool(query: str) -> str:
    """Search the web for current information using a simple web scraping approach"""
    try:
        # Using DuckDuckGo instant answer API as an example
        url = f"https://api.duckduckgo.com/?q={query}&format=json&no_html=1&skip_disambig=1"
        response = requests.get(url, timeout=10)
        data = response.json()

        result = ""
        if data.get('AbstractText'):
            result += f"Summary: {data['AbstractText']}\n"
        if data.get('AbstractURL'):
            result += f"Source: {data['AbstractURL']}\n"

        # If no abstract, try to get related topics
        if not result and data.get('RelatedTopics'):
            result = "Related information:\n"
            for topic in data['RelatedTopics'][:3]:
                if isinstance(topic, dict) and 'Text' in topic:
                    result += f"- {topic['Text']}\n"

        return result if result else f"No specific information found for: {query}"

    except Exception as e:
        return f"Web search failed: {str(e)}"

@tool
def stock_price_tool(symbol: str) -> str:
    """Get current stock price and basic info for a given symbol"""
    try:
        stock = yf.Ticker(symbol.upper())
        info = stock.info
        hist = stock.history(period="1d")

        if hist.empty:
            return f"No data found for symbol: {symbol}"

        current_price = hist['Close'].iloc[-1]
        prev_close = info.get('previousClose', current_price)
        change = current_price - prev_close
        change_pct = (change / prev_close) * 100 if prev_close else 0

        result = f"""
Stock: {symbol.upper()}
Current Price: ${current_price:.2f}
Change: ${change:.2f} ({change_pct:.2f}%)
Company: {info.get('longName', 'N/A')}
Market Cap: ${info.get('marketCap', 0):,}
Volume: {hist['Volume'].iloc[-1]:,}
        """.strip()

        return result

    except Exception as e:
        return f"Stock lookup failed for {symbol}: {str(e)}"

@tool
def calculator_tool(expression: str) -> str:
    """Evaluate mathematical expressions safely"""
    try:
        # Simple whitelist approach for safety
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Expression contains invalid characters"

        # Use eval with restricted environment
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result: {result}"

    except Exception as e:
        return f"Calculation error: {str(e)}"

@tool
def document_search_tool(query: str) -> str:
    """Search through internal documents (simulated)"""
    # In a real implementation, this would connect to your vector store
    documents = {
        "company_policy": "Our company policy emphasizes ethical AI development and responsible deployment.",
        "product_specs": "Our RAG system supports multiple embedding models and vector stores.",
        "meeting_notes": "Last quarter's performance exceeded expectations with 25% growth.",
        "research_papers": "Recent advances in transformer architectures show promising results.",
    }

    # Simple keyword matching (replace with actual vector search)
    results = []
    query_lower = query.lower()

    for doc_id, content in documents.items():
        if any(word in content.lower() for word in query_lower.split()):
            results.append(f"{doc_id}: {content}")

    if results:
        return "\n".join(results)
    else:
        return f"No internal documents found matching: {query}"

class ToolOrchestrator:
    """Manages tool selection and execution with error handling"""

    def __init__(self, tools: List[Tool]):
        self.tools = {tool.name: tool for tool in tools}
        self.execution_history = []

    def execute_tool(self, tool_name: str, tool_input: str, max_retries: int = 2) -> Dict:
        """Execute tool with error handling and retries"""
        execution_record = {
            "tool": tool_name,
            "input": tool_input,
            "timestamp": datetime.now().isoformat(),
            "attempts": 0,
            "success": False,
            "result": None,
            "error": None
        }

        for attempt in range(max_retries + 1):
            execution_record["attempts"] = attempt + 1

            try:
                if tool_name not in self.tools:
                    raise ValueError(f"Tool '{tool_name}' not available")

                tool = self.tools[tool_name]
                result = tool.func(tool_input)

                execution_record["success"] = True
                execution_record["result"] = result
                break

            except Exception as e:
                execution_record["error"] = str(e)
                if attempt < max_retries:
                    print(f"Tool execution failed (attempt {attempt + 1}): {e}")
                    continue
                else:
                    execution_record["result"] = f"Tool execution failed after {max_retries + 1} attempts: {e}"

        self.execution_history.append(execution_record)
        return execution_record

    def get_tool_stats(self) -> Dict:
        """Get statistics about tool usage and success rates"""
        if not self.execution_history:
            return {"total_executions": 0}

        stats = {
            "total_executions": len(self.execution_history),
            "success_rate": sum(1 for record in self.execution_history if record["success"]) / len(self.execution_history),
            "tool_usage": {},
            "common_errors": {}
        }

        for record in self.execution_history:
            tool = record["tool"]
            stats["tool_usage"][tool] = stats["tool_usage"].get(tool, 0) + 1

            if record["error"]:
                error_type = type(record["error"]).__name__
                stats["common_errors"][error_type] = stats["common_errors"].get(error_type, 0) + 1

        return stats

**# SECTION 3: Building a Complete Agentic RAG System**

In [ ]:
print("\n" + "="*60)
print("SECTION 3: COMPLETE AGENTIC RAG SYSTEM")
print("="*60)

class AgenticRAGSystem:
    """Complete agentic RAG system combining all components"""

    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.llm = ChatOpenAI(model=model_name, temperature=0.1)
        self.memory = AgentMemory()
        self.planner = AgentPlanner(self.llm)
        self.callback_handler = AgentCallbackHandler()

        # Initialize tools
        self.tools = [
            web_search_tool,
            stock_price_tool,
            calculator_tool,
            document_search_tool
        ]

        self.orchestrator = ToolOrchestrator(self.tools)

        # Create ReAct agent
        self.agent = self._create_react_agent()
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            max_iterations=5,
            callbacks=[self.callback_handler],
            handle_parsing_errors=True
        )

    def _create_react_agent(self):
        """Create a ReAct agent with custom prompt template"""
        react_prompt = PromptTemplate.from_template("""
You are an intelligent research assistant with access to multiple tools. Your goal is to help users by planning and executing multi-step investigations.

AVAILABLE TOOLS:
{tools}

TOOL DESCRIPTIONS:
{tool_names_and_descriptions}

MEMORY CONTEXT:
{memory_context}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
        """)

        return create_react_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=react_prompt
        )

    def query(self, question: str, use_planning: bool = True) -> Dict:
        """Execute a query with optional planning phase"""
        print(f"\n🎯 Processing query: {question}")

        # Optional planning phase
        if use_planning:
            print("\n📋 Creating execution plan...")
            plan = self.planner.create_plan(question, [tool.name for tool in self.tools])
            print("Plan created:")
            for i, step in enumerate(plan, 1):
                print(f"  {i}. {step.get('description', 'No description')}")

        # Add query to memory
        self.memory.add_working_memory({"type": "query", "content": question})

        # Prepare agent input with memory context
        agent_input = {
            "input": question,
            "memory_context": self.memory.get_context()
        }

        # Execute query
        try:
            result = self.agent_executor.invoke(agent_input)

            # Store result in memory
            self.memory.add_working_memory({
                "type": "result",
                "query": question,
                "answer": result.get("output", "No output")
            })

            return {
                "success": True,
                "answer": result.get("output"),
                "steps": self.callback_handler.get_trace(),
                "tool_stats": self.orchestrator.get_tool_stats()
            }

        except Exception as e:
            error_msg = f"Query execution failed: {str(e)}"
            print(f"❌ {error_msg}")
            return {
                "success": False,
                "error": error_msg,
                "steps": self.callback_handler.get_trace()
            }

    def get_system_status(self) -> Dict:
        """Get comprehensive system status"""
        return {
            "memory_entries": len(self.memory.working_memory),
            "facts_stored": len(self.memory.long_term_facts),
            "tools_available": len(self.tools),
            "tool_stats": self.orchestrator.get_tool_stats(),
            "total_queries": len([m for m in self.memory.working_memory if m["entry"].get("type") == "query"])
        }

**# SECTION 4: Demonstration and Testing**

In [ ]:
print("\n" + "="*60)
print("SECTION 4: DEMONSTRATION AND TESTING")
print("="*60)

# Initialize the agentic RAG system
print("🚀 Initializing Agentic RAG System...")
rag_system = AgenticRAGSystem()

# Test queries demonstrating different capabilities
test_queries = [
    {
        "query": "What is the current stock price of Apple (AAPL) and how does it compare to Microsoft?",
        "description": "Multi-step query requiring tool orchestration"
    },
    {
        "query": "Search for information about quantum computing and calculate what 15% of 1000 would be",
        "description": "Demonstrates tool selection and parallel reasoning"
    },
    {
        "query": "Find our company policy on AI development and search for recent news about AI ethics",
        "description": "Combines internal and external information sources"
    }
]

# Execute test queries
for i, test in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {test['description']}")
    print(f"{'='*60}")

    result = rag_system.query(test["query"])

    if result["success"]:
        print(f"\n✅ Query completed successfully!")
        print(f"Answer: {result['answer']}")
        print(f"Steps taken: {len(result['steps'])}")
    else:
        print(f"\n❌ Query failed: {result['error']}")

    # Show system status after each query
    status = rag_system.get_system_status()
    print(f"\nSystem Status: {status['total_queries']} queries processed, {status['memory_entries']} memory entries")


**# SECTION 5: Advanced Patterns and Error Handling**

In [ ]:
print("\n" + "="*60)
print("SECTION 5: ADVANCED PATTERNS AND ERROR HANDLING")
print("="*60)

class RobustAgenticRAG(AgenticRAGSystem):
    """Enhanced version with advanced error handling and recovery"""

    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        super().__init__(model_name)
        self.fallback_strategies = {
            "tool_failure": self._handle_tool_failure,
            "parsing_error": self._handle_parsing_error,
            "timeout": self._handle_timeout
        }

    def _handle_tool_failure(self, error: Exception, context: Dict) -> str:
        """Handle tool execution failures with fallback strategies"""
        print(f"🔧 Tool failure detected: {error}")

        # Try alternative tools
        failed_tool = context.get("tool")
        if failed_tool == "web_search_tool":
            # Fallback to document search
            return "I'll search our internal documents instead of the web."
        elif failed_tool == "stock_price_tool":
            # Fallback to web search for financial info
            return "I'll search for financial information using web search."

        return "I'll continue with available tools."

    def _handle_parsing_error(self, error: Exception, context: Dict) -> str:
        """Handle agent parsing errors"""
        print(f"📝 Parsing error detected: {error}")
        return "I'll rephrase my approach and try again."

    def _handle_timeout(self, error: Exception, context: Dict) -> str:
        """Handle execution timeouts"""
        print(f"⏱️ Timeout detected: {error}")
        return "I'll provide a partial answer based on available information."

    def query_with_recovery(self, question: str) -> Dict:
        """Execute query with enhanced error recovery"""
        max_attempts = 3

        for attempt in range(max_attempts):
            try:
                result = self.query(question, use_planning=True)
                if result["success"]:
                    return result

                # Apply recovery strategy
                error_type = self._classify_error(result.get("error", ""))
                recovery_message = self.fallback_strategies.get(
                    error_type, lambda e, c: "Retrying with modified approach."
                )(None, {"attempt": attempt})

                print(f"🔄 Recovery attempt {attempt + 1}: {recovery_message}")

            except Exception as e:
                print(f"❌ Attempt {attempt + 1} failed: {e}")
                if attempt == max_attempts - 1:
                    return {
                        "success": False,
                        "error": f"All {max_attempts} attempts failed",
                        "final_error": str(e)
                    }

        return {"success": False, "error": "Maximum recovery attempts exceeded"}

    def _classify_error(self, error_message: str) -> str:
        """Classify error type for appropriate recovery strategy"""
        if "tool" in error_message.lower():
            return "tool_failure"
        elif "parsing" in error_message.lower():
            return "parsing_error"
        elif "timeout" in error_message.lower():
            return "timeout"
        return "unknown"

# Test robust system
print("🛡️ Testing Robust Agentic RAG System...")
robust_rag = RobustAgenticRAG()

# Test with a complex query that might trigger recovery mechanisms
complex_query = "Find the latest stock price for Tesla, search for recent news about electric vehicles, and calculate the percentage change if Tesla stock went from $200 to $250"

print(f"\n🧪 Testing complex query with recovery mechanisms...")
result = robust_rag.query_with_recovery(complex_query)

if result["success"]:
    print("✅ Robust query handling successful!")
    print(f"Answer: {result['answer']}")
else:
    print(f"❌ Even robust handling failed: {result['error']}")

**# SECTION 6: Key Takeaways and Next Steps**

In [ ]:
print("\n" + "="*60)
print("KEY TAKEAWAYS AND NEXT STEPS")
print("="*60)

print("""
🎯 Key Takeaways from Notebook 13.1:

1. AGENT ARCHITECTURE:
   - Agents combine reasoning, planning, memory, and tool use
   - ReAct pattern (Reasoning + Acting) provides structured approach
   - Memory systems enable context persistence across interactions

2. TOOL INTEGRATION:
   - Tools should be focused and reliable
   - Error handling and fallback strategies are crucial
   - Tool orchestration enables complex multi-step reasoning

3. SYSTEM DESIGN:
   - Modularity allows for easy extension and maintenance
   - Monitoring and observability are essential for production
   - Recovery mechanisms improve system reliability

4. PRACTICAL CONSIDERATIONS:
   - Balance autonomy with predictability
   - Plan for failures and edge cases
   - Consider cost and latency implications

🚀 Next Steps:
- Explore multi-step reasoning patterns in Notebook 13.2
- Implement memory systems for long-term context
- Build multi-agent collaboration systems
- Deploy production-ready agentic systems

📊 System Performance Summary:
""")

# Show final system statistics
final_status = robust_rag.get_system_status()
print(f"- Total Queries Processed: {final_status['total_queries']}")
print(f"- Memory Entries: {final_status['memory_entries']}")
print(f"- Tools Available: {final_status['tools_available']}")
print(f"- Tool Success Rate: {final_status['tool_stats'].get('success_rate', 0):.2%}")

print(f"\n✨ Notebook Complete! Ready for multi-step reasoning and memory management.")